In [1]:
import pickle
import usefull_funcs
with open('objs.pkl', 'rb') as f:
    PPC_plants = pickle.load(f)
    
path_to_files = input('Please insert the path to the pvplants folders:')

# dirs = usefull_funcs.list_dir(path_to_files)


Please insert the path to the pvplants folders:/media/spiros/my_data/pvplants


In [2]:
for entry in PPC_plants:
    data=usefull_funcs.parse_ods(entry,path_to_files)
    print(usefull_funcs.find_grid_code(data),'\n')

ARG_Biosar_CaldenesDelOeste_30MWp

Caldenes 

ARG_SterlinWilson_JinkoSolar_CordilleraSolar_93.3MW

Argentina_Cordillera 

AR_CS_Elmya_Cafayate_100MW_3192

Cafayate 

AUS_BNRG_Cedarbrook_75KWp

None 

AU_Biosar_Kiamal_RTU_Test_244.5MWp_3640

None 

BRA_Biosar_BomJesusdaLapaHV_BJL

None 

BRA_Biosar_Guimarania_40.99MWp_Subplant_1

None 

BRA_Biosar_Guimarania_41.53MWp_Subplant_2

None 

BRA_Biosar_Guimarania_82.52MWp_HV_SS

None 

BRA_Biosar_JesusdaLapaBJL04_25MWp

Pirapora 

BRA_Biosar_JesusdaLapaBJL11_25MWp

Pirapora 

BRA_Biosar_PiraporaPhase1_185MWp_HV_SS

None 

BRA_Biosar_PiraporaPhase1_37MWp_Subplant_10

Pirapora 

BRA_Biosar_PiraporaPhase1_37MWp_Subplant_5

Pirapora 

BRA_Biosar_PiraporaPhase1_37MWp_Subplant_6

Pirapora 

BRA_Biosar_PiraporaPhase1_37MWp_Subplant_7

Pirapora 

BRA_Biosar_PiraporaPhase1_37MWp_Subplant_9

Pirapora 

BRA_Biosar_PiraporaPhase2_38MWp_Subplant_2

Pirapora 

BRA_Biosar_PiraporaPhase2_38MWp_Subplant_3

Pirapora 

BRA_Biosar_PiraporaPhase2_38MWp_Subplant_4

Vietnam 

VN_Sungrow_Bach_Khoa_A_Chau2_30MWp_3197

Vietnam 

VN_Sungrow_ERS_30MWp_2956

ERS_30 

VN_Sungrow_ERS_50MWp_2955

Vietnam 

VN_Sungrow_Fuji_40MWp_2864

Vietnam 

VN_Sungrow_GCL_50MWp_2902

Vietnam 

VN_Sungrow_JSC_50MWp_3070

Vietnam 

VN_Sungrow_KimII_50MWp_2866

Vietnam 

VN_Sungrow_KimI_50MWp_2865

Vietnam 

VN_Sungrow_MY_40MWp_3115

Vietnam 

VN_Sungrow_PNP_50MWp_3037

Vietnam 

VN_Sungrow_Quang_Minh_50MWp_3351

Vietnam 

VN_Sungrow_WAN_HONG_2_40MWp_3153

Vietnam 

VN_Sungrow_WAN_HONG_40MWp_3127

Vietnam 

VN_TBEA_Binh_Thuan2_33.1MWp_3226

Vietnam 

VN_VPW_HAM_KIEM_49MWp_3106

HAM_KIEM 

Vietnam_Sungrow_Ayala_168MWp

Vietnam 

Vietnam_Sungrow_DAI_100MWp

Vietnam 

Vietnam_Sungrow_JUWI_30MWp

Vietnam 

Vietnam_Sungrow_JUWI_50MWp

Vietnam 

Vietnam_Sungrow_TTCL_40MWp

Vietnam 

ZA_JUWI_Droogfontein_86.248MWp_3374

SA_Eskom_Droogfontein 

